# A Graph Theory Based Analysis of Urban Forests 
This file serves as a starting point for using graph theory as a representation for urban forests. Here, we will start with Nashville, TN, but the full code base provides modularized components to scale to different cities with comparative analytics. 

1) Downloads OpenStreetMap for a city 
2) Extracts "green" polygons (parks/trees/grass/etc.)
3) Build spatial graph - each green polygon is a node, each edge a potential ecological corridor (lower weight if overlapping with streets) 
4) Computes algebraic connectivity, clustering, path length, 'small worldness', global efficiency 
5) Runs resiliency simulations by removing nodes, exporting geospatial layers 
6) Figures to summarize the entwork 
  

In [2]:
!pip install ipykernel

In [3]:
# necessary packages 
%pip install geopandas networkx osmnx pandas numpy scipy matplotlib tqdm shapely pyproj fiona rtree joblib

Note: you may need to restart the kernel to use updated packages.


In [4]:

import math, os, random, sys
from dataclasses import dataclass
from typing import Tuple, Optional

import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
from joblib import Parallel, delayed
from scipy.sparse.linalg import eigsh
from shapely.geometry import Point, Polygon, LineString, MultiPolygon
from shapely.ops import unary_union
from tqdm import tqdm

We start first by representing urban forests as a network. 

| Network Element | Urban Forest Equivalent | 
|---|---|
| Node | Individual trees, patches of green space, or microhabitats |
| Edge | Row 2, Cell 2 | 
| Cluster | Groups of trees forming micro-ecosystems or green patches | 
| Shortcut | Linear green corridors, green roofs, street trees that connect distant parks | 

The purpose of this is to give us a structured way of evaluating current urban layout. According to {INSERT UR SOURCE HERE}, the benefits of urban greenspaces range from reducing heat island effects to psychological and health benefits for inhabitants. 



In [ ]:
"""
    Small container for tunable parameters used across the pipeline.

place : str
    Place name query for OSMnx (used to geocode boundary & fetch data)
out_dir : str
    Directory where all outputs (gpkg, plots, csv) are written
crs_projected : str
    Projected CRS used for distance/area ops (autodetected later)
patch_min_area_m2 : float
    Minimum polygon area (m²) to keep as a "green patch"
street_buffer_m : float
    Buffer (meters) around streets to discount corridor costs when overlapped
edge_cost_tau : float
    Decay constant controlling how edge weights decrease with effective cost
connect_dist_m : float
    Base neighborhood distance (meters) to consider patch–patch connections
grid_side_m : float
    Grid side for optional gridding (not used downstream here)
random_seed : int
    Seed for reproducible random operations
"""

@dataclass
class Config:
    place: str = "Nashville, Tennessee, USA"
    out_dir: str = "outputs_nashville"
    crs_projected: str = "EPSG:3857"  #
    patch_min_area_m2: float = 200.0
    street_buffer_m: float = 12.0
    edge_cost_tau: float = 150.0
    connect_dist_m: float = 120.0
    grid_side_m: float = 500.0
    random_seed: int = 42